In [ ]:
from process import process, status
from transforms import master_transform
import pandas as pd
import matplotlib
pd.options.display.float_format = '{:,.2f}'.format
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

## Load and Aggregate All the Data

In [ ]:
mt = master_transform.Loader()
df_raw = mt.load_all()

## Process a Label the Data

In [ ]:
pr = process.Process()
df_process = pr.process(df_raw)
pr.__del__()

### Check for the last time the data was updated

In [ ]:
st = status.LastUpdate()
st.last_update(df_process)

## Total Wealth Time Series

In [ ]:
piv_ccy = pd.DataFrame(pd.pivot_table(df_process, values='Amount', index=['Currency', 'Date'], columns=[], aggfunc=sum).to_records())
piv = pd.DataFrame(pd.pivot_table(df_process, values='Amount', index=['Date'], columns=[], aggfunc=sum).to_records())

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
for ccy in piv_ccy.Currency.unique():
    ax.plot(piv_ccy[piv_ccy.Currency == ccy]['Date'], piv_ccy[piv_ccy.Currency == ccy]['Amount'].cumsum(), label = ccy)

ax.legend()
ax.grid()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
ax.xaxis.set_minor_formatter(mdates.DateFormatter("%Y-%m"))

### Wealth Breakdown by Asset

In [ ]:
pd.DataFrame(pd.pivot_table(df_process, values='Amount', index=['AccountType', 'Currency'], columns=[], aggfunc=sum).to_records())\
        .sort_values('Amount', ascending=False, ignore_index=True)

### Wealth Breakdown by Account

In [ ]:
pd.DataFrame(pd.pivot_table(df_process, values='Amount', index=['AccountType', 'Currency', 'Account'], columns=[], aggfunc=sum).to_records())

## Spending Split

In [ ]:
import plotly.express as px
df_expenses = df_process[(df_process.FullType != 'Savings') 
                         & (df_process.YearToDate < 1)  
                         & (df_process.FacingAccount == '')  
                         & (df_process.Amount < 0) 
                         & (df_process.FullType != 'Intra-Account Transfert') 
                         & (df_process.Currency == 'GBP')]
df_expenses.Amount = (-1) * df_expenses.Amount

In [ ]:
fig = px.sunburst(df_expenses[df_expenses.Account == '20-26-77 13105881'], path=['FullType', 'MemoMapped'], values='Amount')
fig.show()

In [ ]:
fig = px.sunburst(df_expenses[df_expenses.Account == '30-92-90 34061868'], path=['FullSubType', 'MemoMapped'], values='Amount')
fig.show()

In [ ]:
fig = px.sunburst(df_expenses, path=['FullType', 'MemoMapped'], values='Amount')
fig.show()

In [ ]:
pd.DataFrame(pd.pivot_table(df_process, values='Amount', index=['AccountType', 'Account'], columns=[], aggfunc=sum).to_records())\
.to_csv('account_wealth.csv', index=False)

In [ ]:
per_day = df_process.set_index('Date').groupby(pd.Grouper(freq='d'))['Amount'].sum()
df_process['MemoDetailed'] = df_process.MemoMapped + ": " + df_process.Amount.map('{:,.0f}'.format)

df_agg = df_process[df_process.FacingAccount == ''].groupby(['Date', 'MemoDetailed']).agg({'Amount':sum})
g = df_agg['Amount'].groupby(level=0, group_keys=False)
res = g.apply(lambda x: x.sort_values(ascending=False))
per_day_label =pd.DataFrame(res.to_frame().to_records()).groupby(pd.Grouper(key='Date', freq='d'))['MemoDetailed']\
            .apply(lambda x: '<br>'.join(x.unique()))

In [ ]:
import plotly.graph_objects as go
import numpy as np
fig = go.Figure(data=go.Scatter(x=per_day.index, y=np.cumsum(per_day.values), hovertext=per_day_label))

fig.update_layout(
    title="Total Wealth",
    xaxis_title="Date")

fig.show()

## Break down the last year

In [ ]:
pd.pivot_table(df_process[(df_process.YearToDate < 1) & (df_process.FacingAccount == '') & (df_process.Currency == 'GBP')], 
               values='Amount', 
               index=['FullType'], 
               columns=['AdjustedYear', 'AdjustedMonth'], 
               aggfunc=sum,
               margins=True,
               fill_value = 0).sort_values('All', ascending=False, axis=0).drop('All', axis=0)

## Project Wealth

In [ ]:
from projection import projection as pj

In [ ]:
r=range(0, 11)
(w, w_low, w_up, w_low_ex, w_up_ex) = pj.project_full(pd.read_csv('account_wealth_projection.csv'), r)

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
ax.plot(piv['Date'], piv['Amount'].cumsum())

ax.plot([piv['Date'].values[-1] + np.timedelta64(365*i,'D') for i in r], w, '-o', color='b', label='Expected Wealth')
ax.fill_between([piv['Date'].values[-1] + np.timedelta64(365*i,'D') for i in r], w_low, w_up, color='b', alpha=.2, label='Less Likely')
ax.fill_between([piv['Date'].values[-1] + np.timedelta64(365*i,'D') for i in r], w_low_ex, w_up_ex, color='b', alpha=.1, label='Most Likely')

ax.grid()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
ax.xaxis.set_minor_formatter(mdates.DateFormatter("%Y-%m"))

## Inter-Account Flows

In [ ]:
df_internal = df_process[(df_process.FacingAccount != '') & (df_process.Amount > 1000) & (df_process.YearToDate < 5)]
df_internal = pd.DataFrame(pd.pivot_table(df_internal, values='Amount', index=['Account', 'FacingAccount'], columns=[], aggfunc=sum).to_records())

In [ ]:
map_list = list(set(sorted(list(set(df_internal.Account.values)) + list(set(df_internal.FacingAccount.values)))))

In [ ]:
import plotly.graph_objects as go
# override gray link colors with 'source' colors
opacity = 0.4


fig = go.Figure(data=[go.Sankey(
    valueformat = ".0f",
    valuesuffix = "£",
    # Define nodes
    node = dict(
      pad = 15,
      thickness = 15,
      line = dict(color = "black", width = 0.5),
      label =  map_list,
    ),
    # Add links
    link = dict(
      source =  [map_list.index(s) for s in df_internal['Account'].values],
      target =  [map_list.index(s) for s in df_internal['FacingAccount'].values],
      value =  df_internal['Amount']
))])

fig.update_layout(font_size=10)
fig.show()